In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import AE
from models import DNN
import numpy as np
import pandas as pd

In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 500 #200,500,1000
dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1

# Import data

In [4]:
data_r=pd.read_csv('data/GDSCexpression.csv',index_col=0)
label_r=pd.read_csv('data/GDSClabel.csv',index_col=0)

In [5]:
label_r=label_r.fillna(1)

# Your is gene-cell, mine is cell-gene

In [6]:
len(set(data_r.index))

727

In [7]:
data = data_r
label = label_r.iloc[:,10]
scaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
data = scaler.fit_transform(data)

In [8]:
print(np.std(data))
print(np.mean(data))

0.9999999999999997
6.316789844568889e-19


In [9]:
data.shape

(789, 11833)

In [10]:
label_r.shape

(789, 139)

# Split test train

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)

In [12]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(789, 11833)
(789,)
(631, 11833) (631,)
(158, 11833) (158,)


In [13]:
print(X_train.max())
print(X_train.min())

23.78951596267762
-17.277718272630676


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [15]:
trainData = torch.FloatTensor(X_train).to(device)
testData = torch.FloatTensor(X_test).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
allData = torch.FloatTensor(data).to(device)

# construct TensorDataset
train_dataset = TensorDataset(trainData, trainData)
test_dataset = TensorDataset(testData, testData)
all_dataset = TensorDataset(allData, allData)

trainDataLoader1 = DataLoader(dataset=train_dataset, batch_size=200, shuffle=False)
trainDataLoaderall = DataLoader(dataset=all_dataset, batch_size=200, shuffle=False)

# The model

In [16]:
# Initialization

#autoencoder = AE(dim=train_dataset.shape[1]).to(device)
autoencoder = AE(dim_au_in = X_train.shape[1],dim_au_out=dim_au_out).to(device)
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
#loss1
loss_func = nn.SmoothL1Loss().to(device)
#loss2
#loss_func = nn.BCELoss()
loss_train = np.zeros((epochs, 1))

In [ ]:
# Train autoencoder

for epoch in range(epochs):
    # 不需要label，所以用一个占位符"_"代替
    for batchidx, (x, _) in enumerate(trainDataLoaderall):
        x.requires_grad_(True)
        # encode and decode 
        decoded, encoded = autoencoder(x)
        # compute loss
        print(encoded.shape, decoded.shape)
        loss = loss_func(decoded, x)      
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item()))

In [18]:
torch.save(autoencoder.state_dict(), 'saved/models/AE_GDSC.pkl')

In [20]:
# extract features
_, encodedTrainData = autoencoder(trainData)
featureTensor = encodedTrainData.double()
feature = featureTensor.detach().cpu().numpy()

In [21]:
print(feature.shape)

(631, 512)


In [22]:
feature

array([[-16.69282913,   0.4927558 , -11.76840973, ...,  25.36968613,
         -1.19588506,  -3.05758238],
       [ 12.47170448,  -6.22012472, -32.64635086, ..., -11.72289944,
         11.10695744, -14.05883694],
       [-10.22851944,   7.51775646, -42.93748856, ..., -20.94465637,
        -33.18747711,   6.23425484],
       ...,
       [  2.99668527, -17.52783966,  -0.15471686, ...,  -2.72013927,
        -38.31910706,  15.50337124],
       [ -0.09494959,   0.19647354, -29.87499046, ..., -16.57998466,
        -15.16943169, -10.62462997],
       [-25.3997345 , -10.5040102 ,   0.81054193, ..., -54.55727386,
        -22.51106071, -16.26926994]])

In [23]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(feature, Y_train.values)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [24]:
_,testFeature = autoencoder(testData)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [25]:
r2_score(lasso,Y_test)

-3.1212509541498186

In [26]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 631 samples in 0.007s...
[t-SNE] Computed neighbors for 631 samples in 0.263s...
[t-SNE] Computed conditional probabilities for sample 631 / 631
[t-SNE] Mean sigma: 132.380762
[t-SNE] KL divergence after 250 iterations with early exaggeration: 71.162323
[t-SNE] KL divergence after 300 iterations: 1.585068


In [27]:
tsne_results

array([[  1.2682492, -11.414175 ],
       [  4.1123514,   4.6177115],
       [  7.306512 ,  -0.3178884],
       ...,
       [ -5.1699767,   0.7598758],
       [  9.604777 ,  -2.1241179],
       [-13.449348 ,   1.7129223]], dtype=float32)

In [28]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.savefig("saved/figrues/tsne_ae_gdsc.png")

In [29]:
EPOCH = 500

In [30]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(feature).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [31]:
b_data.tensors

(tensor([[-16.6928,   0.4928, -11.7684,  ...,  25.3697,  -1.1959,  -3.0576],
         [ 12.4717,  -6.2201, -32.6464,  ..., -11.7229,  11.1070, -14.0588],
         [-10.2285,   7.5178, -42.9375,  ..., -20.9447, -33.1875,   6.2343],
         ...,
         [  2.9967, -17.5278,  -0.1547,  ...,  -2.7201, -38.3191,  15.5034],
         [ -0.0949,   0.1965, -29.8750,  ..., -16.5800, -15.1694, -10.6246],
         [-25.3997, -10.5040,   0.8105,  ..., -54.5573, -22.5111, -16.2693]],
        device='cuda:0'),
 tensor([4.1360e-03, 2.4651e-01, 1.0000e+00, 1.6831e-01, 2.4553e-01, 4.5574e-02,
         5.6766e-01, 6.5160e-03, 5.1101e-02, 1.0000e+00, 1.0000e+00, 1.3382e-01,
         8.5730e-03, 2.0783e-02, 7.1891e-02, 2.6463e-02, 2.0267e-01, 1.6792e-01,
         2.2742e-01, 5.1173e-01, 5.5870e-02, 9.3305e-02, 3.9400e-03, 9.9018e-02,
         1.0000e+00, 7.3892e-02, 1.1134e-02, 8.2544e-02, 1.1754e-02, 1.4302e-01,
         1.0000e+00, 1.0000e+00, 2.4001e-02, 3.9460e-03, 7.5210e-03, 4.9375e-01,
         7.

In [32]:
b_data

In [33]:
dim_dnn_in

512

In [34]:
# Initialization DNN model

predictor = DNN(dim_dnn_in, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
loss_func = nn.MSELoss().to(device)
#loss2-sigmoid
#loss_func = nn.BCELoss()
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [35]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 

# Save model
torch.save(predictor.state_dict(), 'saved/models/DNN_GDSC.pkl')

Epoch:  0


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=se

Epoch: 0001, Training loss=0.14173484
Epoch:  1


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=se

Epoch: 0002, Training loss=0.14206359
Epoch:  2


...

Epoch: 0498, Training loss=0.10444779
Epoch:  498


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=se

Epoch: 0499, Training loss=0.10444133
Epoch:  499


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=se

Epoch: 0500, Training loss=0.10446230


In [36]:
# Get tesing feature
_,testFeature = autoencoder(testData)

In [37]:
testpredict = predictor(testFeature)

In [38]:
testpredict

tensor([[0.2738],
        [0.2857],
        [0.2624],
        [0.1795],
        [0.2214],
        [0.2446],
        [0.1997],
        [0.2364],
        [0.2220],
        [0.2769],
        [0.2001],
        [0.2178],
        [0.2473],
        [0.2393],
        [0.2148],
        [0.2587],
        [0.2176],
        [0.2256],
        [0.1555],
        [0.2596],
        [0.2343],
        [0.2307],
        [0.1995],
        [0.2171],
        [0.2028],
        [0.2465],
        [0.2174],
        [0.2384],
        [0.1566],
        [0.2523],
        [0.1876],
        [0.2440],
        [0.2397],
        [0.2244],
        [0.2086],
        [0.1659],
        [0.2329],
        [0.2234],
        [0.2245],
        [0.1897],
        [0.2303],
        [0.1702],
        [0.1894],
        [0.2569],
        [0.2420],
        [0.2231],
        [0.1904],
        [0.2307],
        [0.2286],
        [0.2354],
        [0.2350],
        [0.2817],
        [0.3100],
        [0.2895],
        [0.1947],
        [0

In [39]:
from sklearn.metrics import mean_squared_error

In [40]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

-128.53321292905125

In [41]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)

0.13347006273282727